In [3]:
import json
import os
import shutil

# Путь к корневой папке
root_dir = "dataset_fire_smoke"

# Папка для YOLO данных
yolo_dir = os.path.join(root_dir, "yolo")
data_dir = os.path.join(yolo_dir, "data")

# Список папок для обработки
folders = {"474_fire_val": "val", "475_fire_train": "train"}

# Словарь для преобразования названий классов в ID
class_map = {"fire": 0, "smoke": 1}

def convert_coco_to_yolo(json_path, source_images_dir, target_images_dir, target_labels_dir):
    # Создаём папки для изображений и аннотаций
    os.makedirs(target_images_dir, exist_ok=True)
    os.makedirs(target_labels_dir, exist_ok=True)

    # Читаем JSON файл
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    # Создаём словарь изображений (id -> filename, width, height)
    images = {img["id"]: {"file_name": img["file_name"], "width": img["width"], "height": img["height"]}
              for img in coco_data["images"]}

    # Создаём словарь аннотаций по image_id
    annotations_by_image = {}
    for ann in coco_data["annotations"]:
        image_id = ann["image_id"]
        if image_id not in annotations_by_image:
            annotations_by_image[image_id] = []
        annotations_by_image[image_id].append(ann)

    # Обрабатываем каждое изображение
    for image_id, img_info in images.items():
        # Извлекаем только имя файла
        image_filename = os.path.basename(img_info["file_name"])
        source_image_path = os.path.join(source_images_dir, image_filename)
        target_image_path = os.path.join(target_images_dir, image_filename)
        
        if not os.path.exists(source_image_path):
            print(f"Изображение {source_image_path} не найдено, пропускаем.")
            continue

        # Копируем изображение в новую папку
        shutil.copy2(source_image_path, target_image_path)

        # Имя файла аннотации
        txt_filename = os.path.splitext(image_filename)[0] + ".txt"
        txt_path = os.path.join(target_labels_dir, txt_filename)

        # Проверяем, есть ли аннотации для этого изображения
        if image_id not in annotations_by_image:
            print(f"Нет аннотаций для {image_filename}, пропускаем.")
            continue

        # Открываем файл для записи
        with open(txt_path, 'w') as txt_file:
            for ann in annotations_by_image[image_id]:
                # Извлекаем bounding box и категорию
                bbox = ann["bbox"]  # [x_min, y_min, width, height]
                category_id = ann["category_id"]
                
                # Находим имя класса по category_id
                class_name = next(cat["name"] for cat in coco_data["categories"] if cat["id"] == category_id)
                class_id = class_map.get(class_name)
                if class_id is None:
                    print(f"Неизвестный класс: {class_name}")
                    continue

                # Преобразуем в YOLO формат
                img_width = img_info["width"]
                img_height = img_info["height"]
                
                x_min, y_min, width, height = bbox
                x_center = (x_min + width / 2) / img_width
                y_center = (y_min + height / 2) / img_height
                norm_width = width / img_width
                norm_height = height / img_height

                # Записываем в файл
                txt_file.write(f"{class_id} {x_center} {y_center} {norm_width} {norm_height}\n")

        print(f"Созданы аннотации для {image_filename} в {txt_path}")

# Создание структуры папок и обработка данных
for folder, split in folders.items():
    json_path = os.path.join(root_dir, folder, "annotations", "instances_default.json")
    source_images_dir = os.path.join(root_dir, folder, "images", "val" if folder == "474_fire_val" else "train")
    target_images_dir = os.path.join(data_dir, split, "images")
    target_labels_dir = os.path.join(data_dir, split, "labels")

    if not os.path.exists(json_path):
        print(f"Файл {json_path} не найден!")
        continue

    print(f"Обработка {folder} -> {split}...")
    convert_coco_to_yolo(json_path, source_images_dir, target_images_dir, target_labels_dir)

# Создание data.yaml
data_yaml_path = os.path.join(yolo_dir, "data.yaml")
with open(data_yaml_path, 'w') as f:
    f.write(f"""\
train: {os.path.join(data_dir, "train", "images")}
val: {os.path.join(data_dir, "val", "images")}
nc: 2
names: ['fire', 'smoke']
""")

print(f"Создан файл {data_yaml_path}")
print("Преобразование завершено!")

Обработка 474_fire_val -> val...
Созданы аннотации для 000016.jpg в dataset_fire_smoke\yolo\data\val\labels\000016.txt
Созданы аннотации для 000019.jpg в dataset_fire_smoke\yolo\data\val\labels\000019.txt
Созданы аннотации для 000024.jpg в dataset_fire_smoke\yolo\data\val\labels\000024.txt
Созданы аннотации для 000027.jpg в dataset_fire_smoke\yolo\data\val\labels\000027.txt
Созданы аннотации для 000028.jpg в dataset_fire_smoke\yolo\data\val\labels\000028.txt
Созданы аннотации для 000029.jpg в dataset_fire_smoke\yolo\data\val\labels\000029.txt
Созданы аннотации для 000036.jpg в dataset_fire_smoke\yolo\data\val\labels\000036.txt
Созданы аннотации для 000046.jpg в dataset_fire_smoke\yolo\data\val\labels\000046.txt
Созданы аннотации для 000059.jpg в dataset_fire_smoke\yolo\data\val\labels\000059.txt
Созданы аннотации для 000063.jpg в dataset_fire_smoke\yolo\data\val\labels\000063.txt
Неизвестный класс: person
Созданы аннотации для 000070.jpg в dataset_fire_smoke\yolo\data\val\labels\000070